In [1]:
import sys
import os
sys.path.append('..')
import edlib
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import operator
from string import ascii_uppercase
from itertools import groupby
from copy import deepcopy
import bisect

import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2

%load_ext autoreload
%autoreload 2

# Recruitment of centromeric reads from cen6

In [2]:
import pandas as pd
import numpy as np
import bisect

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
fasta_index_fn = "../data/rel3/rel3.fastq.gz.fai"

In [4]:
fasta_index = pd.read_csv(fasta_index_fn,
                          sep='\t',
                          names=["NAME", "LENGTH", "OFFSET", "LINEBASES", "LINEWIDTH", "QUALOFFSET"])

In [5]:
fasta_index.shape

(28449385, 6)

Total length

In [6]:
total_length = np.sum(fasta_index.LENGTH)
genome_len = 3.1 * 10**9
total_coverage = total_length / genome_len
print(total_length / 10**9, total_coverage)

367.2312828 118.461704129


N50:

In [7]:
sorted_lengths = np.sort(fasta_index.LENGTH)[::-1]
n50_index = bisect.bisect_left(np.cumsum(sorted_lengths), total_length / 2)
sorted_lengths[n50_index]

52930

In [8]:
max_short_len = 5000



Number of short reads


In [9]:
short_reads_index = fasta_index.LENGTH < max_short_len
np.sum(short_reads_index)

19283008



Coverage of human genome with short reads:


In [10]:
total_short_read_len = np.sum(fasta_index.LENGTH[short_reads_index])
total_short_read_len / genome_len

5.7382193370967745

Min long len

In [11]:
min_long_len = 50000



Number of long reads


In [12]:
long_reads_index = fasta_index.LENGTH > min_long_len
np.sum(long_reads_index)

1999007

Total length of long reads and coverage of human genome with long reads:

In [13]:
total_long_read_len = np.sum(fasta_index.LENGTH[long_reads_index])
total_long_read_len / 10**9, total_long_read_len / genome_len

(192.61815650599999, 62.134889195483872)

In [14]:
%reset -f 

Second paragraph

In [15]:
import os
import pandas as pd
import numpy as np

In [16]:
centromeric_reads_dir = "../data/centroFlyeMono_results_cen6/centromeric_reads"

! faidx {centromeric_reads_dir}/centromeric_reads.fasta > /dev/null



In [17]:
centromeric_reads_index_fn = \
    os.path.join(centromeric_reads_dir, 'centromeric_reads.fasta.fai')
centromeric_reads_index = pd.read_csv(centromeric_reads_index_fn,
                                     sep='\t',
                                     names=["NAME", "LENGTH", "OFFSET", "LINEBASES", "LINEWIDTH", "QUALOFFSET"])

In [18]:
nreads = centromeric_reads_index.shape[0]
nreads

6558

In [19]:
np.sum(centromeric_reads_index.LENGTH)

267720128

In [20]:
long_len = 50000

nlong = np.sum(centromeric_reads_index.LENGTH > long_len)

longest_len = np.max(centromeric_reads_index.LENGTH)
print(nlong, longest_len)

1621 530278


In [21]:
%reset -f

# Transforming reads into monoreads

In [22]:
import sys
sys.path.append('../centroFlye_repo/scripts')

import networkx as nx
import numpy as np
import subprocess

from debruijn_graph import DeBruijnGraph, get_frequent_kmers
from sd_parser import SD_Report, get_stats
from mono_error_correction import error_correction
from utils.bio import read_bio_seqs

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
sd_report_fn = '../data/centroFlyeMono_results_cen6/string_decomposer_report/decomposition.tsv'
monomers_fn = '../data/D6Z1/D6Z1_monomers.fasta'

sd_report = SD_Report(SD_report_fn=sd_report_fn, monomers_fn=monomers_fn)

In [24]:
get_stats(sd_report.monostrings, verbose=True, return_stats=False)

Number of translations: 6558
Min length = 17
Mean length = 238.32464165904238
Max length = 3195
Total length = 1562933
#(%) Gap symbols = 34303 (0.021947837815184657)
#Gap runs = 7375


In [25]:
monoreads = error_correction(sd_report.monostrings, verbose=True, inplace=False)

Stats for uncorrected reads
Number of translations: 6558
Min length = 17
Mean length = 238.32464165904238
Max length = 3195
Total length = 1562933
#(%) Gap symbols = 34303 (0.021947837815184657)
#Gap runs = 7375

Stats for filtered reads
Number of translations: 6537
Min length = 17
Mean length = 237.9418693590332
Max length = 3195
Total length = 1555426
#(%) Gap symbols = 33437 (0.02149700467910399)
#Gap runs = 7297

Stats for trimmed+filtered reads
Number of translations: 6537
Min length = 0
Mean length = 235.28805262352762
Max length = 3195
Total length = 1538078
#(%) Gap symbols = 20807 (0.013527922511082013)
#Gap runs = 6190

Stats for cut_gaprich_reads+trimmed+filtered reads
# cut reads = 124
# total cut parts = 188
Number of translations: 6412
Min length = 0
Mean length = 228.91593886462883
Max length = 2662
Total length = 1467809
#(%) Gap symbols = 5989 (0.004080231147240547)
#Gap runs = 3193

Stats for hor_corrected+cut_gaprich_reads+trimmed+filtered reads
Number of translation

HOR graph for MinMultShortKMer = 100, MinMultShortKMer = 1000

In [26]:
def hor_graph(min_mult, k=3):
    frequent_kmers, frequent_kmers_read_pos = get_frequent_kmers(sd_report.monostrings, k=k, min_mult=min_mult)
    db = DeBruijnGraph(k=k)
    db.add_kmers(frequent_kmers, coverage=frequent_kmers)
    
    hors, _ = db.get_contigs()
    print(hors)
    hor_graph = nx.MultiDiGraph()
    for st, end, m_ind, edge_data in db.graph.edges(data=True, keys=True):
        edge_str = edge_data['edge_kmer']
        hor_graph.add_edge(edge_str[:k-1], edge_str[-k+1:], m_ind, label=edge_str)
    
    !mkdir ../data/cen6_hor_graph
    dot_fn = f'../data/cen6_hor_graph/hor_db_{min_mult}.dot'
    pdf_fn = f'../data/cen6_hor_graph/hor_db_{min_mult}.pdf'
    nx.drawing.nx_pydot.write_dot(hor_graph, dot_fn)
    !dot -Tpdf -Efontsize=10 {dot_fn} -o {pdf_fn} 
    return db

In [27]:
hor_graph(1000)
hor_graph(5000)
pass

['ABCDE', 'ABFGHIJKLMNOPQRA', 'DEFGHIJKLMNOPQRA', 'FGHIJKLMNOPQRAB', 'FGHIJKLMNOPQRADE']
mkdir: cannot create directory ‘../data/cen6_hor_graph’: File exists
['FGHIJKLMNOPQRABCDE', 'FGHIJKLMNOPQRAB']
mkdir: cannot create directory ‘../data/cen6_hor_graph’: File exists


# Constructing iterative de Bruijn graphs of monoreads

In [28]:
import networkx as nx

from collections import defaultdict, Counter
from debruijn_graph import DeBruijnGraph, iterative_graph, scaffolding, read2scaffolds, cover_scaffolds_w_reads, extract_read_pseudounits, polish

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
def statistics_gapfree_kmers(monoreads, k, gap_symb='?'):
    cnt_all, cnt_gapfree = 0, 0
    for monoread in monoreads.values():
        for i in range(len(monoread)-k+1):
            kmer = monoread[i:i+k]
            if gap_symb not in kmer:
                cnt_gapfree += 1
            cnt_all += 1
    return cnt_all, cnt_gapfree, cnt_gapfree / cnt_all

statistics_gapfree_kmers(monoreads, k=400)

(249119, 222639, 0.8937054178926537)

# Quality assessment of cen6

In [30]:
# ! python ../centroFlye_repo/scripts/ext/tandemQUAST/tandemquast.py \
#     -l cen6-0.1.3 \
#     -r ../data/centroFlyeMono_results_cen6/centromeric_reads/centromeric_reads.fasta \
#     -o ../data/centroFlyeMono_results_cen6/tandemQUAST_report \
#     ../data/centroFlyeMono_results_cen6/centroFlyeMono_cen6/polishing/scaffold_0/scaffold_0.fasta
